In [19]:
#imports
from fugashi import Tagger
import pandas as pd
import torch
from collections import Counter
from transformers import AutoModel, AutoTokenizer
import unicodedata
from datasets import load_dataset

import os
import re
import numpy as np 
from sklearn.metrics import accuracy_score

import transformers

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

device

'cpu'

In [38]:
#data_files = {"train": "train_split.csv", "test": "test_split.csv", 'val': 'val_split.csv'}
#dataset = load_dataset('universal_dependencies', 'ja_gsd')
train_split = dataset['train']
test_split = dataset['test']
val_split = dataset['validation']


In [44]:
print(len(train_split))
print(len(test_split))
print(len(val_split))
val_split['text']

7027
543
501


['ただし、50周年ソングに変更後は、EDも歌つきのものが使われた。',
 '私は初めてだったんだけど思っていたよりも魚は新鮮でした。',
 'セントラル・リーグ審判員の水落朋大は実兄。',
 '多彩なライブアクトとともに、祝日前の渋谷の夜を盛り上げてくれそうだ。',
 '今作品では大会でのパラメータUPが非常に大きく、ALL999への育成は回復アイテムを投与しつつ、冬眠、復活を繰り返し大会に連続して出すという方法が取られた。',
 'ゲーム入力のレスポンスをよくするためには、これを1秒間に通常30回以上行う必要があり、実際に得られたデータの回数と値はゲームコントローラ内の抵抗や回路上のノイズやCPUスピード、ゲームコントローラ全体の容量からくるRC回路・時定数に依存する。',
 '姉と同じ先生だったので、話やすかったです。',
 '同僚教師のすみれと彩はそんな夕子と意気投合し、問題を解決するため行動を共にする。',
 'ほかの複数の信者も女性宅近くで付きまとい行為をしていたとみられ,統一教会の組織的な関与の有無を慎重に調べている。',
 '背中に背負ったブースターを使って空中飛行を行う。',
 'また、最新版のウェブブラウザを利用していても、OSやコンポーネント側にセキュリティホールが存在すれば、そこを突かれる可能性がある。',
 'しかも、熱々の料理はとても美味しかったです。',
 '取り調べ段階では黙秘を貫き“全ては裁判官の前で話す”としていた被告は,犯行を認めたものの教祖への逮捕状発布の経緯に関し“捜査当局の不手際,失態”と元同僚らを非難,法廷は一時元警視の独壇場となった。',
 '今回、日本のメディアもこのテストに参加することができたので、最新バージョンの出来具合をレポートしていきたい。',
 '大正期から、ノンフィクション、推理小説などを訳した。',
 '市内のユダヤ人は1942年までにその全員が逃亡するか、強制収容所に送致されるかのいずれかであった。',
 'ほとんどは浅海生だが、水深数百mほどの深海まで生息するものもいる。',
 '価格に見合う満足感を感じます。',
 '一方B国では輸出により小麦の量が減るので、小麦の価格は上がる。',
 'インターナショナル・マネジメント・グループをはじめとする投資家によって所有しているが、中でも元テニス選手ピート

In [71]:
#load kanji data
kanji_df = pd.read_json("kanji.json")
tagger = Tagger('-Owakati')
kanji_df = kanji_df.drop(['strokes', 'grade', 'freq', 'jlpt_old', 'jlpt_new', 'meanings', 'wk_radicals', 'wk_readings_kun', 'wk_readings_on', 'wk_meanings', 'wk_level'], axis=0)

In [109]:
kanji_df#['周']

,一,二,九,七,人,入,八,力,十,三,...,視,謁,謹,賓,贈,辶,逸,難,響,頻
readings_on,"[いち, いつ]","[に, じ]","[きゅう, く]",[しち],"[じん, にん]","[にゅう, じゅ]",[はち],"[りょく, りき, りい]","[じゅう, じっ, じゅっ]","[さん, ぞう]",...,[し],[えつ],[きん],[ひん],"[そう, ぞう]",[],[],[なん],[きょう],[ひん]
readings_kun,"[ひと-, ひと.つ]","[ふた, ふた.つ, ふたたび]","[ここの, ここの.つ]","[なな, なな.つ, なの]","[ひと, -り, -と]","[い.る, -い.る, -い.り, い.れる, -い.れ, はい.る]","[や, や.つ, やっ.つ, よう]",[ちから],"[とお, と]","[み, み.つ, みっ.つ]",...,[],[],[つつしむ],[],[おくる],[しんにょう],[しんにょう],"[かたい, むずかしい]",[ひびく],[]


In [103]:
output = []
word_list = []
pos_list = []
text = "きのう、りんごをたべた"
#text = 'たべた'
#text = 'いち'
tagger.parse(text)
# => '麩 菓子 は 、 麩 を 主材 料 と し た 日本 の 菓子 。'
for word in tagger(text):
    output.append(word.feature.lemma)
    word_list.append(word.char_type)
    pos_list.append(word.pos)
    print(word.feature.orth)
    #print(word.feature.lemma, sep=' ')
    # "feature" is the Unidic feature data as a named tuple
#word_list = ' '.join(word_list)
#print(output)

きのう
、
りんご
を
たべ
た


In [102]:
for word in word_list:
    print(word)
    for readings_on in kanji_df.loc['readings_on']:
        #print(readings_on)
        for reading in readings_on:
            if unicodedata.normalize('NFC', word) == u reading:
                print(reading)
                #print(wordreading)
            #else:
                #print(word, reading)

SyntaxError: invalid syntax (2079165740.py, line 6)

In [41]:
if 'いち' == 'いち':
    print('いち')

いち


In [108]:

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# input text
# use pos tagger
# get new input
# use hmm to access correctedness 

#WIll TO DO
#What do you need to be successful?

#text = input("input text: ")
output = []
word_list = []
pos_list = []
text = "きのう、りんごをたべた"
#target = "昨日、林檎を食べた"
#tokenize sentence(s) to words
#The tagger things
tagger.parse(text)
for word in tagger(text):
    #tokenize word to char
    #n! amount of combinations 
    #print all combinations of chars
        # (if) compare all combinations of char to on and kun readings
            #ignore particles for now
            #store kanji (and leftovers)
            
        #need to pair/add new dataset
        #HMM (probability, haven't refreshed on project yet, but I think it should follow this) 
        #for combinations in new dataset (how often does it show up... たべ田 probably shows up less than 食べた)
            #ideally check best option w/ prob of previous word existing before it (can use tags for HMM, word.pos)
            #    #return best one 
#build our own test sentences?...

Suffix Tree
I like to eat food

Use Kanji Lookup table to apply on HuggingFace data to get the kana only version 

In [11]:
class MultiLabelDataset(torch.utils.data.Dataset):

    def __init__(self, text, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.targets = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

class BERTClass(torch.nn.Module):
    def __init__(self, NUM_OUT):
        super(BERTClass, self).__init__()
                   
        self.l1 = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")#BertModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 32)
        self.classifier = torch.nn.Linear(32, NUM_OUT)
        self.dropout = torch.nn.Dropout(0.5)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        output = self.classifier(pooler)
#         pooler = torch.nn.Tanh()(pooler)
#         pooler = self.dropout(pooler)
        
        output = self.softmax(output)
        return output
    
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

def train(model, training_loader, optimizer):
    model.train()
    for data in tqdm(training_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss
    
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for data in tqdm(testing_loader):
            targets = data['targets']
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            outputs = torch.sigmoid(outputs).cpu().detach()
            fin_outputs.extend(outputs)
            fin_targets.extend(targets)
    return torch.stack(fin_outputs), torch.stack(fin_targets)

In [12]:
#Vars
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
NUM_OUT = 7 # binary task
LEARNING_RATE = 2e-05

training_data = MultiLabelDataset(train_X, train_y, tokenizer, MAX_LEN)
test_data = MultiLabelDataset(test_X, test_y, tokenizer, MAX_LEN)

train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }    

training_loader = torch.utils.data.DataLoader(training_data, **train_params)
testing_loader = torch.utils.data.DataLoader(test_data, **test_params)

NameError: name 'train_X' is not defined

In [13]:
model = BERTClass(NUM_OUT)
model.to(device)    

optimizer_bert = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'device' is not defined